import

In [ ]:
import pandas as pd
import numpy as np
import math
import random
import sys

from itertools import combinations

file = 'processed_data.csv'
df =pd.read_csv(r"./data/%s" % file)


Input variables

In [ ]:
selected_variables = ['ApplicationType', 'binned_CreditScore', 'LoanGoal', 'binned_RequestedAmount', 
                      'binned_FirstWithdrawalAmount', 'binned_MonthlyCost', 'binned_NoOfTerms', 'binned_NumberOfOffers']
controllable_variables =  ['binned_FirstWithdrawalAmount', 'binned_MonthlyCost', 'binned_NoOfTerms', 'binned_NumberOfOffers']
target = 'Selected'

In [ ]:
import re #for sorting

data = pd.DataFrame()
data['Z'] = df[target]
data['notZ'] = 1 - data['Z']
new_controllable_variables=[]

# from https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


controllable_feature_list =[] # double array, consisting of lists of related (exlusive) features



def one_hot_encoding(column_name: str):
    """ For every unique value in the original column a seperate column will be added in the binary dataframe called 'data',
    and when the column only has 2 values, only 1 column will be created,
    where for both the name of the column is the name of the column of origin and the selected value combined,
    where for both 1 represents the value in the original column was equal to the selected value and 0 otherwise,
    the function returns for binary columns the encoder and for the other columns, the names of the original values.
    """
    
    # There is only 1 column needed to decifer the original value if the original column had only 2 values
    if len(df[column_name].unique()) == 2:
        binary_list = []
        value = df[column_name].unique()[0]
        encoding = [[df[column_name].unique()[0],1],[df[column_name].unique()[1],0]]
        for i in df[column_name]:
            if i == value:
                binary_list.append(1)
            else:
                binary_list.append(0)
        data[column_name +"_"+ str(value)] = binary_list
        if column_name in controllable_variables:
            new_controllable_variables.append(column_name +"_"+ str(value))
            cfl=[column_name +"_"+ value]
            controllable_feature_list.append(cfl)
        return encoding
        
    if len(df[column_name].unique()) > 2:
        cfl=[]
        values=df[column_name].unique()
        sorted_values=sorted(values,key=natural_keys)
        for value in sorted_values:
            binary_list = []
            for i in df[column_name]:
                if i == value:
                    binary_list.append(1)
                else:
                    binary_list.append(0)
            data[column_name +"_"+ value] = binary_list
            if column_name in controllable_variables:
                new_controllable_variables.append(column_name +"_"+ value)
                cfl.append(column_name +"_"+ value)
        if cfl!=[]:
            controllable_feature_list.append(cfl)
        return list(df[column_name].unique())
    
# Execute the one-hot encoding and retrieve the encoding scheme
encoding_scheme = {}

for i in selected_variables:
    encoding_scheme[i] = one_hot_encoding(i)
    
#Set the variables to one-hot encoded column_names and pop the target variable
new_selected_variables = list(data.columns)
new_selected_variables.pop(0)
new_selected_variables.pop(0)
print(new_controllable_variables)


Functions and Classes for the algorithm

In [ ]:

from collections import defaultdict
from re import X


rank=defaultdict(int)


class Node:
    def __init__(self, itemName, itemSet, parentNodeList,d):
        self.itemName = itemName
        self.itemSet = itemSet
        self.lsupp = 0
        self.parentlist = parentNodeList
        self.children = set ()
        self.depth=d;


    def display(self, ind=1):
        print('  ' * ind, self.itemSet, ' ', self.lsupp)
        for child in list(self.children):
            child.display(ind+1)

def generateNextLevelTree(treeNode, root, level, RHS, dataset):
    if treeNode.depth <level-1: 
        for c in treeNode.children:
            generateNextLevelTree(c, root, level, RHS, dataset)
    else: 
        if treeNode.depth==level-1:
            for c1 in treeNode.children:
                for c2 in treeNode.children:
                    if c1==c2: continue
                    itemSet=c1.itemSet.union(c2.itemSet)
                    if (len(itemSet)==0):
                        print("non-empty itemset expected")
                        sys.exit()
                    if contain(root,root,itemSet)!=None:
                        continue # similar node has been created already
                    
                    stop = False
                
                    parentNodeList = list()
                    for v in itemSet:
                        itemSetminV =itemSet.copy()
                        itemSetminV.remove(v)
                        if len(itemSetminV)==0:
                             continue;
                        n = contain(root,root,itemSetminV)
                        if n== None:
                            stop = True
                            break
                        else:
                            parentNodeList.append(n)
                    if stop: continue
                    
                    itemName= itemSet - c1.itemSet
                    itemName=itemName.pop()
                    if not c1 in parentNodeList:
                        parentNodeList.append(c1)
                    newNode = Node(itemName,itemSet,parentNodeList,c1.depth+1)
                    newNode.lsupp =local_support(itemSet,RHS,dataset)
                    c1.children.add(newNode)


def contain(treeNode, root, itemSet: set):
    if itemSet == set():
        # should be impossible
        treeNode.display(10)
        sys.exit()
    if treeNode == root:
        for c in treeNode.children:
            n=contain(c,root,itemSet)
            if n!= None:
                return n
    else:
        if treeNode.itemSet == itemSet:
            return treeNode
        else: 
            if treeNode.itemName not in itemSet:
                return None
            for c in treeNode.children:
               n=contain(c,root,itemSet)
               if n!= None :
                  return n
    return None


def prune(T,minsup):
    if T.depth>0 and T.lsupp < minsup:
        for parent in T.parentlist:
            parentchildren=parent.children
            parentchildren.discard(T)
    else:
        for c in T.children.copy():
            prune(c,minsup)

#observation 2 from paper by Li et al.
def pruneSameSupport(T):
    if T.depth>0 :
        same=True
        for parent in T.parentlist:
            if T.lsupp!=parent.lsupp:
                same=False
                break
        if same:
            parentchildren=parent.children
            parentchildren.discard(T)
        else:
           for c in T.children.copy():
            pruneSameSupport(c) 
    else:
        for c in T.children.copy():
            pruneSameSupport(c)

def removeNode(T, itemSet):
    if T.itemSet==itemSet:
        for parent in T.parentlist:
            parentchildren=parent.children
            parentchildren.discard(T)
        exit
    else:
        for c in T.children.copy():
            removeNode(c,itemSet)

def generateAssocationRules(T,k,RHS,dataset):
    LHS_AR=[]
    if T.depth==k:
        if association_test(list(T.itemSet),RHS,dataset):
            LHS_AR.append(list(T.itemSet))
    else:
        i=0
        for c in T.children:
            LHS_AR=LHS_AR+generateAssocationRules(c,k,RHS,dataset)
    return LHS_AR
  
#RHS plays the role of z
def support_counter(var_list,RHS, dataset):
    #Create for the variables in the list a mask where the values all should be equal to 1
    p = 1
    for v in var_list:
        p = p & (dataset[v] ==1)
    
    supp_p_z = dataset[p][RHS].sum()
    supp_p_not_z = dataset[p][RHS].count() - supp_p_z
    
    not_p = ~p
    supp_not_p_z = dataset[not_p][RHS].sum()
    supp_not_p_not_z = dataset[not_p][RHS].count() - dataset[not_p][RHS].sum()
    return supp_p_z, supp_p_not_z, supp_not_p_not_z, supp_not_p_z

# RHS plays the role of z
def local_support(var_list, RHS, dataset) -> float:
    """ Uses the support_counter() function to calculate the local support"""
    
    supp_p_z, supp_p_not_z, supp_not_p_not_z, supp_not_p_z = support_counter(var_list, RHS, dataset)
    supp_z = supp_p_z + supp_not_p_z
    l_supp = supp_p_z/supp_z 
    
    return l_supp

# check support of rule var_list -> RHS, where RHS plays the role of z below
def computeMetrics (var_list, RHS, dataset):
    supp_p_z, supp_p_not_z, supp_not_p_not_z, supp_not_p_z = support_counter(var_list, RHS, dataset)
    supp = supp_p_z / dataset.shape[0]
    supp_p = supp_p_z + supp_p_not_z
    supp_z = supp_p_z + supp_not_p_z 
    conf = supp / supp_p
    lift = supp_p_z / (supp_p*supp_z)
    return supp, conf, lift

def association_test(var_list: list, RHS, dataset) -> bool:
    """ An association is significant if the oddsratio is with 95% confidence higher than 1. In this function the association 
    between the set of variables in var_list and the target variable RHS is tested. Returns True when there is a significant
    association and False otherwise
    """
    supp_p_z, supp_p_not_z, supp_not_p_not_z, supp_not_p_z = support_counter(var_list, RHS, dataset)
    #Haldane-Anscombe correction for when one of the support is 0 (the correction is just adding 0.5 to all)
    supp_p_z, supp_p_not_z= supp_p_z + 0.5, supp_p_not_z + 0.5
    supp_not_p_not_z, supp_not_p_z = supp_not_p_not_z + 0.5 , supp_not_p_z + 0.5
    
    #95% confidence -> z' = 1.96, odr = oddsratio(p -> z) and lb = lower bound
    odr =(supp_p_z * supp_not_p_not_z)/(supp_p_not_z * supp_not_p_z)
    
    lb_or= math.exp(np.log(odr)- (1.96*(math.sqrt((1/supp_p_z) + (1/supp_p_not_z) + (1/supp_not_p_z) + (1/supp_not_p_not_z)))))
    if lb_or > 1:
        return True
    else:
        return False

def find_exclusive_variables(LHS:list, X:list, dataset, min_l_supp = 0) -> list:
    """
    Requires input of the lefthand side of the association rule and the list of variables which can be exclusive from the 
    lefthand side (X). For each element in X it is tested whether the element is exclusive from the LHS list. The function 
    returns the set of variables that are exclusive in list E. 
    """
    
    E = []
    P = LHS
    if P==[]:
        return []

    p = 1
    for v in LHS:
        p = p & (dataset[v] ==1)        
    

    for variable in X:
        Q = variable
        if Q not in P:
            q = (dataset[Q] == 1)
            mask_p_q = p & q
            mask_not_p_q = (~p) & q
            supp_p_q = dataset[mask_p_q][Q].count()
            supp_not_p_q = dataset[mask_not_p_q][Q].count()
            if supp_p_q <= min_l_supp:
                E.append(Q)
            elif supp_p_q <= min_l_supp:
                E.append(Q)  
    return E

def create_fair_dataset(dataset, LHS_set:list, RHS, C:list):
    """
    Matches tuples and returns n11, n12, n21, n22
    """

    all_relevant_vars = C + [RHS] + LHS_set
    
    n12, n21 = 0, 0
    
    P = LHS_set

    df_C = dataset[all_relevant_vars].groupby(C)
    for name, df_c in df_C:
        p =1
        for i in P:
            p = p & (df_c[i] == 1)
        df_p = df_c[all_relevant_vars][p]
        df_not_p = df_c[all_relevant_vars][~p]
        if df_p.count()[0]+df_not_p.count()[0]!=df_c.count()[0]:
            print("missing rows counted")
            sys.exit()
        count = min (df_p.count()[0],df_not_p.count()[0])
        if count==0:
            continue
        df_p_sample = df_p.sample(count)
        df_not_p_sample = df_not_p.sample(count)
        pz_count= df_p_sample[RHS].sum()
        pnotz_count = count - df_p_sample[RHS].sum()
        notpz_count= df_not_p_sample[RHS].sum()
        notpnotz_count= count - df_not_p_sample[RHS].sum()
        if (pz_count <= notpnotz_count):
            n12+=pz_count
            n21+=notpz_count
        else:
            n12+=notpnotz_count
            n21+=pnotz_count


    return n12, n21




In [ ]:
min_l_supp = 0.01 #Based of page 18
RHS='Z'
RHS_set = [RHS]
max_k = 6


In [ ]:
#1
pRc = []   # stores the positive causal association rules found with fair data sets

Rc_or = dict() # odds ratio

Rc_supp = dict() # support

Rc_conf = dict() # confidence

Rc_lift = dict() # lift

RcCHM = [] # stores the causal association rules found with orginal data sets and odds ratios/ Cochlan-Haenszel-Mantel metric

#2
main_root = Node('Null',{},[],0)

for v in new_selected_variables:
    n=Node(v,{v},[main_root],1)
    main_root.children.add(n)
    rank[v]=data[v].sum()
    n.lsupp =local_support({v},RHS,data)


#4
prune(main_root,min_l_supp)

X = [ c.itemName for c in main_root.children]

I = set()
for variable in X:
    #If the function does not return True:
    if not association_test([variable],RHS, data):
        I.add(variable)
        

#7 
k = 1

In [ ]:


#8
while k <= max_k:
    found= False
#9 LHS_AR stores the sets of variables of the LHS of the k-th level
    LHS_AR= generateAssocationRules(main_root,k,RHS,data)

#11
    for LHS in LHS_AR:
        E = find_exclusive_variables(LHS, X, data)
#12     
        C = [variable for variable in X if variable not in I 
                                         if variable not in E
                                         if variable not in LHS]

#13
        n12, n21 = create_fair_dataset(data, LHS, RHS, C)
#14 calculate Odds Ratio, page 10 if zero > count = 1 to evade infinite odds ratios
        if n21 == 0:
            n21 = 1
        if n12 == 0:
            n12 = 1
            
        O_ratio = n12/n21
        # if lowerbound odds ratio (lb_or) > 1 with confidence 955 (z = 1.96)
        lb_or = math.exp(np.log(O_ratio)- (1.96*(math.sqrt((1/(n12)) + (1/(n21))))))
        if lb_or > 1:
#15         
            found= True
            pRc.append(LHS)
#16
            supp, conf, lift= computeMetrics(LHS,RHS,data)
            print ('------------------')
            print(LHS, '-> Z is a positive CR')
            print ('odds ratio', O_ratio)
            print ('support: ', supp)
            print ('confidence: ', conf)
            print ('lift: ', lift)
            print ('------------------')
            Rc_or[tuple(LHS)]=O_ratio
            Rc_supp[tuple(LHS)]=supp
            Rc_conf[tuple(LHS)]=conf
            Rc_lift[tuple(LHS)]=lift
 
            
        if found: # prune LHS from prefix tree
            removeNode(main_root,set(LHS))
            n=contain(main_root,main_root,set(LHS))
            if n!=None:
                print("Error: tree contains removed node!")
                n.display(1)
                sys.exit()
       
            
    if not found:
        print ("No CR rules found at level ",k)

    # No nodes should be added after the max tree depth has been reached
    if k == max_k:
        break
    
    print('\n'+'adding level ', k, 'nodes to the tree') 

#19
    generateNextLevelTree(main_root,main_root,k,RHS,data)  
    prune(main_root,min_l_supp)
    pruneSameSupport(main_root)
    k += 1



In [ ]:
print('postive causal rule ; odds ratio ; confidence ; support ; lift ' )
for LHS in pRc:
    print(LHS, '-> Z is a CR ; ', Rc_or[tuple(LHS)], ';', Rc_conf[tuple(LHS)], ';', Rc_supp[tuple(LHS)], ';', Rc_lift[tuple(LHS)] ,'\n' )
    


In [ ]:
min_l_supp = 0.01 #Based of page 18
RHS='notZ'
RHS_set = [RHS]
max_k = 6

In [ ]:
#1
nRc = []   # stores the negataive causal association rules found with fair data sets

nRc_or = dict() # odds ratio

nRc_supp = dict() # support

nRc_conf = dict() # confidence

nRc_lift = dict() # lift


#2
nmain_root = Node('Null',{},[],0)

for v in new_selected_variables:
    n=Node(v,{v},[nmain_root],1)
    nmain_root.children.add(n)
    rank[v]=data[v].sum()
    n.lsupp =local_support({v},RHS,data)


#4
prune(nmain_root,min_l_supp)

X = [ c.itemName for c in nmain_root.children]

I = set()
for variable in X:
    #If the function does not return True:
    if not association_test([variable],RHS, data):
        I.add(variable)
        

#7 
k = 1

In [ ]:


#8
while k <= max_k:
    found= False
#9 LHS_AR stores the sets of variables of the LHS of the k-th level
    LHS_AR= generateAssocationRules(nmain_root,k,RHS,data)

#11
    for LHS in LHS_AR:
        E = find_exclusive_variables(LHS, X, data)
#12     
        C = [variable for variable in X if variable not in I 
                                         if variable not in E
                                         if variable not in LHS]

#13
        n12, n21 = create_fair_dataset(data, LHS, RHS, C)
#14 calculate Odds Ratio, page 10 if zero > count = 1 to evade infinite odds ratios
        if n21 == 0:
            n21 = 1
        if n12 == 0:
            n12 = 1
            
        O_ratio = n12/n21
        # if lowerbound odds ratio (lb_or) > 1 with confidence 955 (z = 1.96)
        lb_or = math.exp(np.log(O_ratio)- (1.96*(math.sqrt((1/(n12)) + (1/(n21))))))
        if lb_or > 1:
#15         
            found= True
            nRc.append(LHS)
#16
            supp, conf, lift= computeMetrics(LHS,RHS,data)
            print ('------------------')
            print(LHS, '-> !Z is a negative CR')
            print ('odds ratio', O_ratio)
            print ('support: ', supp)
            print ('confidence: ', conf)
            print ('lift: ', lift)
            print ('------------------')
            nRc_or[tuple(LHS)]=O_ratio
            nRc_supp[tuple(LHS)]=supp
            nRc_conf[tuple(LHS)]=conf
            nRc_lift[tuple(LHS)]=lift

            
        if found:
            removeNode(main_root,set(LHS))
            n=contain(main_root,main_root,set(LHS))
            if n!=None:
                print("Error: tree contains removed node!")
                n.display(1)
                sys.exit()
       
            
    if not found:
        print ("No CR rules found at level ",k)

    # No nodes should be added after the max tree depth has been reached
    if k == max_k:
        break
   
    print('\n'+'adding level ', k, 'nodes to the tree') 

#19
    generateNextLevelTree(main_root,main_root,k,RHS,data)  
    prune(main_root,min_l_supp)
    pruneSameSupport(main_root)
    k += 1





In [ ]:
print('negative causal rule ; odds ratio ; confidence ; support ; lift ' )
for LHS in nRc:
    print(LHS, '-> !Z is a CR ; ', nRc_or[tuple(LHS)], ';', nRc_conf[tuple(LHS)], ';', nRc_supp[tuple(LHS)], ';', nRc_lift[tuple(LHS)] , '\n' )
    


In [ ]:
# return true if a positive causal rule can be applied to the case, so the case does not (fully) satisfy the LHS and LHS can be completely changed (is fully controllable)
def evaluatepositivecausalrule(LHS, case)-> bool:
    for I in LHS:
        if case[I]!=1 and I in new_controllable_variables:
            return True
    return False    

# return true if a negative causal rule applies to the case with the positive features in pimpact, so the case + pimpact satisfies the LHS and the LHS can be completely changed (is fully controllable)
def evaluatenegativecausalrule(LHS, pimpact, case)-> bool:
    for I in LHS:
        if case[I]!=1 and I not in pimpact:
            return False
        if  not (I in new_controllable_variables):
            return False
    return True    

def positiveimpact(LHS,case)->list:
    plist=[]
    for I in LHS:
        if case[I]!=1 and I in new_controllable_variables:
            plist.append(I)
    return plist   

def negativeimpact(LHS,pimpact,case)->list:
    nlist=[]
    for I in LHS:
        if (case[I]==1 or I in pimpact) and I in new_controllable_variables:
            nlist.append(I)
    return nlist   

def exclusive(f):
    l=[]
    for cfl in controllable_feature_list:
        if f in cfl:
            l = cfl[:]
            l.remove(f)
            return l
    return l

def areexclusive(f1,f2):
    if f1==f2: 
        return False
    for cfl in controllable_feature_list:
        if f1 in cfl and f2 in cfl:
            return True
    return False

def distance(f1,f2):
    for cfl in controllable_feature_list:
        if f1 in cfl and f2 in cfl:
            return abs(cfl.index(f1)-cfl.index(f2))
    return -1

def alternative(f,case,Fpos,Fneg):
    for fp in Fpos:
        if areexclusive(f,fp):
            return fp
    E=exclusive(f)
    lowestdist=len(E) # distance
    closestf=None
    l=[ f for f in E if (f not in Fneg)]

    for fn in l:
        d=distance(f,fn) 
        if d<lowestdist:
            lowestdist=d
            closestf=fn
    return closestf

def prevalternative(f,case):
    E=exclusive(f)
    for e in E:
        if case[e]==1:
            return e
    return None

def resolveconflicts(Fpos,Fprev):
    toremove=[]
    for f1 in Fpos:
        for f2 in Fpos:
            if areexclusive(f1,f2):
                for f in Fprev:
                    if areexclusive(f,f1):
                        if distance(f,f1)<distance(f,f2):
                            toremove.append(f2)
                        else:
                            toremove.append(f1)
    Fposnew= [ f for f in Fpos if (f not in toremove) ]
    return Fposnew

def findoutcomealternativecase(dataset,case,Fneg,Falt,Fprev,Fpos):
    mask=1
    for v in new_selected_variables:
        if v in Fneg or v in Falt or v in Fprev or v in Fpos:
            continue
        if case[v]==1:
            mask=mask & (dataset[v]==1)
        else:
            mask=mask & (data[v]==0)
    for f in Fneg:
        mask=mask & (dataset[f]==0)
    for f in Falt:
        mask=mask & (dataset[f]==1)
    for f in Fprev:
        mask=mask & (dataset[f]==0)
    for f in Fpos:
        mask=mask & (dataset[f]==1)
    alternativeoutcomes=dataset[mask]
    print ('number of masked data rows ', dataset[mask].count()[0])
    print ('number of times alternative case is already present', dataset[mask]['Z'].count())
    print ('number of times alternative case has positive outcome', dataset[mask]['Z'].sum())
    print ('number of times alternative case has negative outcome', (dataset[mask]['Z']==0).sum()) 
    return dataset[mask]['Z'].count(), dataset[mask]['Z'].sum(), (dataset[mask]['Z']==0).sum()

In [ ]:
k=0
cols=['Index', 'Pt', 'Nt', 'Alt', 'PAlt', 'NAlt']
df_cases = pd.DataFrame(columns = cols)

for index, case in data.iterrows():
    if (case['Z']==1): continue # only process fail cases
    k+=1
    pimpact=[]
    nimpact=[]
    for LHS in pRc:
        if evaluatepositivecausalrule(LHS,case):
            pimpact.extend(positiveimpact(LHS,case))
    for LHS in nRc:
        if evaluatenegativecausalrule(LHS,pimpact,case):
            nimpact.extend(negativeimpact(LHS,pimpact,case))
    print ('Case ', k, ': ', case)
    Fneg=nimpact
    Fpos=pimpact
    pimps2=[ p for p in Fpos if (p  in Fneg)]
    if len(pimps2)>0:
        print ('Negative feature appears also as positive feature, please rerun to compute alternative fair data set')
        sys.exit()
    Fpos=[ p for p in Fpos if (p not in Fneg)]
    Falt=[] # features are in conflict with features in LHS of negative causal rule satisfied by c
    for n in Fneg:
        falt=alternative(n,case,Fpos,Fneg)
        if falt==None:
            print ('Error: no alternative feature found for ', n)
            exit()
        Falt.append(falt)
        print ('Negative Repair: replace ', n, ' with ', falt)
    Fprev=[] # features satisfied by c that are in conflict with features introduced by LHS of positive causal rule
    for p in Fpos:
        fprev=prevalternative(p,case)
        if fprev==None:
            print ('Error: no alternative feature found for ', n)
            exit()
        Fprev.append(fprev)
    Fposnew=resolveconflicts(Fpos,Fprev) # conflicts can only be resolved if Fprev is known to compute the lowest repair distance
    for p in Fposnew:
        fprev=prevalternative(p,case)
        if fprev==None:
            print ('Error: no alternative feature found for ', n)
            exit()
        print ('Positive Repair: replace ', fprev, ' with ', p)

    (alt_cases, pos_alt, neg_alt)= findoutcomealternativecase(data,case,Fneg,Falt,Fprev,Fposnew)
    df_cases = df_cases.append({'Index' : index, 'Pt' : len(Fposnew), 'Nt' : len(Fneg), 'Alt':alt_cases, 'PAlt':pos_alt, 'NAlt':neg_alt },
        ignore_index = True)

df_cases.to_csv("repair_results.csv", sep=";", index=False)
df_cases.describe().to_csv("desc.csv", sep=";", index=False)

